In [ ]:
import pandas as pd

from google.colab import files

In [ ]:
uploaded = files.upload()

Saving trimet_stopevents_2022-12-07.html to trimet_stopevents_2022-12-07.html


In [ ]:
from bs4 import BeautifulSoup
from io import StringIO

with open('trimet_stopevents_2022-12-07.html', 'r') as file:
    soup = BeautifulSoup(file, 'html.parser')



In [ ]:
from datetime import datetime, timedelta

required_columns = {'trip_number', 'vehicle_number', 'arrive_time', 'location_id', 'ons', 'offs'}

# Parse tables
tables = soup.find_all('table')
print(f"Found {len(tables)} HTML <table> tags")

valid_dfs = []
for i, table in enumerate(tables):
    try:
        df = pd.read_html(StringIO(str(table)))[0]
        if required_columns.issubset(set(df.columns)):
            valid_dfs.append(df)
    except Exception:
        continue

# Combine all relevant tables
if not valid_dfs:
    raise ValueError("No valid stop event tables found.")

df = pd.concat(valid_dfs, ignore_index=True)
print(f"Total number of DataFrame has {len(df)} rows")

# Rename trip_number to trip_id
df = df.rename(columns={'trip_number': 'trip_id'})


base_date = datetime(2022, 12, 7)
df['tstamp'] = df['arrive_time'].apply(lambda x: base_date + timedelta(seconds=int(x)))

# Step 4: Create final DataFrame
stops_df = df[['trip_id', 'vehicle_number', 'tstamp', 'location_id', 'ons', 'offs']]



Found 1442 HTML <table> tags
Total number of DataFrame has 93912 rows


In [ ]:

# Analysis
num_vehicles = stops_df['vehicle_number'].nunique()
print(f"Number of vehicles: {num_vehicles}")


Number of vehicles: 158


In [ ]:

num_locations = stops_df['location_id'].nunique()
print(f"Number of unique stop locations: {num_locations}")



Number of unique stop locations: 4354


In [ ]:
min_tstamp = stops_df['tstamp'].min()
print(f"Min timestamp: {min_tstamp}")

max_tstamp = stops_df['tstamp'].max()
print(f"Max timestamp: {max_tstamp}")



Min timestamp: 2022-12-07 04:02:29
Max timestamp: 2022-12-08 02:37:41


In [ ]:
num_boarded = (stops_df['ons'] >= 1).sum()
percent_boarded = (num_boarded / len(stops_df)) * 100

print(f"Stop events with at least one boarding: {num_boarded}")
print(f"Percentage of stop events with boarding: {percent_boarded}%")

Stop events with at least one boarding: 19858
Percentage of stop events with boarding: 21.14532754067638%


In [ ]:
location_id = 6913
location_df = stops_df[stops_df['location_id'] == location_id]

num_stops_at_location = len(location_df)
print(f"Total stops: {num_stops_at_location}")


Total stops: 15


In [ ]:
buses_at_location = location_df['vehicle_number'].nunique()
print(f"Unique buses: {buses_at_location}")


Unique buses: 5


In [ ]:
num_with_boarding = (location_df['ons'] >= 1).sum()
percent_with_boarding = (num_with_boarding / num_stops_at_location) * 100
print(f"Stops with at least one boarding: {num_with_boarding}")
print(f"Percentage with boarding: {percent_with_boarding}%")

Stops with at least one boarding: 2
Percentage with boarding: 13.333333333333334%


In [ ]:
vehicle_id = 4062
vehicle_df = stops_df[stops_df['vehicle_number'] == vehicle_id]
num_stops_by_vehicle = len(vehicle_df)

print(f"Total stops: {num_stops_by_vehicle}")


Total stops: 68


In [ ]:
total_boarded = vehicle_df['ons'].sum()
print(f"Total passengers boarded: {total_boarded}")

Total passengers boarded: 26


In [ ]:
total_deboarded = vehicle_df['offs'].sum()
print(f"Total passengers deboarded: {total_deboarded}")

Total passengers deboarded: 26


In [ ]:
num_vehicle_boarding = (vehicle_df['ons'] >= 1).sum()
percent_vehicle_boarding = (num_vehicle_boarding / num_stops_by_vehicle) * 100
print(f"Percentage of stops with boarding: {percent_vehicle_boarding}%")

Percentage of stops with boarding: 16.176470588235293%


In [ ]:
!pip install scipy

In [ ]:
from scipy.stats import binomtest

In [ ]:
total_events = len(stops_df)
total_boarded_events = (stops_df['ons'] >= 1).sum()
system_boarding_rate = total_boarded_events / total_events
print(f"System-wide boarding rate: {system_boarding_rate}")


System-wide boarding rate: 0.21145327540676379


In [ ]:
vehicle_stats = stops_df.groupby('vehicle_number').agg(
    total_stops=('ons', 'count'),
    stops_with_boarding=('ons', lambda x: (x >= 1).sum())
).reset_index()
vehicle_stats['total_stops'] = vehicle_stats['total_stops'].astype(int)
vehicle_stats['stops_with_boarding'] = vehicle_stats['stops_with_boarding'].astype(int)

vehicle_stats['boarding_rate'] = vehicle_stats['stops_with_boarding'] / vehicle_stats['total_stops']
vehicle_stats['p_value'] = vehicle_stats.apply(
    lambda row: binomtest(
        int(row['stops_with_boarding']),
        int(row['total_stops']),
        system_boarding_rate,
        alternative='two-sided'
    ).pvalue,
    axis=1
)
biased_vehicles = vehicle_stats[vehicle_stats['p_value'] < 0.05].sort_values('p_value')
print(f"Vehicles with biased boarding patterns (p < 0.05):")
print(biased_vehicles[['vehicle_number', 'boarding_rate', 'p_value']])

Vehicles with biased boarding patterns (p < 0.05):
     vehicle_number  boarding_rate   p_value
113            3915       0.173716  0.017249
70             3530       0.173611  0.028077
125            3963       0.167901  0.033011
103            3733       0.180536  0.043074
88             3634       0.171149  0.045715


In [ ]:
uploaded = files.upload()

Saving trimet_relpos_2022-12-07.csv to trimet_relpos_2022-12-07.csv


In [ ]:
from scipy.stats import ttest_ind


In [ ]:
gps_df = pd.read_csv("trimet_relpos_2022-12-07.csv")
print(gps_df.columns)

all_relpos = gps_df['RELPOS'].dropna().values  # remove missing if any

# Step 2: Group by vehicle
vehicle_groups = gps_df.dropna(subset=['RELPOS']).groupby('VEHICLE_NUMBER')

results = []

# Step 3: For each vehicle, run a t-test comparing its RELPOS values to overall
for vehicle_id, group in vehicle_groups:
    vehicle_relpos = group['RELPOS'].values
    if len(vehicle_relpos) >= 5:  # minimum data to be statistically valid
        t_stat, p_val = ttest_ind(vehicle_relpos, all_relpos, equal_var=False)
        results.append((vehicle_id, p_val))

# Step 4: Filter biased vehicles (p < 0.005)
biased_gps_vehicles = pd.DataFrame(results, columns=['VEHICLE_NUMBER', 'p_value'])
biased_gps_vehicles = biased_gps_vehicles[biased_gps_vehicles['p_value'] < 0.005].sort_values('p_value')

# Show results
print("Vehicles with GPS bias (p < 0.005):")
print(biased_gps_vehicles)

Index(['TIMESTAMP', 'VEHICLE_NUMBER', 'RELPOS'], dtype='object')
Vehicles with GPS bias (p < 0.005):
     VEHICLE_NUMBER   p_value
90             3638  0.000000
109            3804  0.000000
157            4305  0.000000
132            4024  0.004008


In [ ]:
from scipy.stats import chi2_contingency


In [ ]:
total_ons = stops_df['ons'].sum()
total_offs = stops_df['offs'].sum()

# Step 2: Group by vehicle
vehicle_totals = stops_df.groupby('vehicle_number').agg(
    vehicle_ons=('ons', 'sum'),
    vehicle_offs=('offs', 'sum')
).reset_index()

# Step 3: Apply Chi-squared test for each vehicle
results = []
for _, row in vehicle_totals.iterrows():
    vehicle_ons = row['vehicle_ons']
    vehicle_offs = row['vehicle_offs']

    # Create 2x2 contingency table
    contingency = [
        [vehicle_ons, vehicle_offs],
        [total_ons - vehicle_ons, total_offs - vehicle_offs]
    ]

    chi2, p, dof, expected = chi2_contingency(contingency)
    results.append((row['vehicle_number'], p))

# Step 4: Filter for significant bias (p < 0.05)
biased_boarding = pd.DataFrame(results, columns=['vehicle_number', 'p_value'])
biased_boarding = biased_boarding[biased_boarding['p_value'] < 0.05].sort_values('p_value')

# Show result
print("Vehicles with biased boarding/deboarding ratios (p < 0.05):")
print(biased_boarding)

Vehicles with biased boarding/deboarding ratios (p < 0.05):
    vehicle_number   p_value
78            3576  0.018783
28            3056  0.030134
